Download a pretrained codellama model. This is finetuned for python [CodeLlama-7B-Instruct-GGUF](https://huggingface.co/TheBloke/CodeLlama-7B-Instruct-GGUF)

In [4]:
!wget -O codellama-7b-instruct.Q8_0.gguf https://huggingface.co/TheBloke/CodeLlama-7B-Instruct-GGUF/resolve/main/codellama-7b-instruct.Q8_0.gguf?download=true


--2024-05-03 22:20:34--  https://huggingface.co/TheBloke/CodeLlama-7B-Instruct-GGUF/resolve/main/codellama-7b-instruct.Q8_0.gguf?download=true
Resolving huggingface.co (huggingface.co)... 3.160.5.76, 3.160.5.109, 3.160.5.25, ...
Connecting to huggingface.co (huggingface.co)|3.160.5.76|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b7/f6/b7f68e1c7abee7ca052604f565465a073f450bb16d60428836f9cca7677ddeb1/2126a5b9e8576ebea8889792ec5e459423935daae17ac0ffdfbedb39d222a20e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27codellama-7b-instruct.Q8_0.gguf%3B+filename%3D%22codellama-7b-instruct.Q8_0.gguf%22%3B&Expires=1715034035&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNTAzNDAzNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iNy9mNi9iN2Y2OGUxYzdhYmVlN2NhMDUyNjA0ZjU2NTQ2NWEwNzNmNDUwYmIxNmQ2MDQyODgzNmY5Y2NhNzY3N2RkZWIxLzIxMjZhNWI5ZTg1NzZlYmVhODg

We are going to upload this into our s3 bucket for serving.

In [1]:
!pip install awscli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 29.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 196.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 223.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 133.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.85
    Uninstalling botocore-1.31.85:
      Successfully uninstalled botocore-1.31.85
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.7.0
    Uninstalling s3transfer-0.7.0:
      Successfully uninstalled s3transfer-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.28.85 req

Ensure you have NOOBAA_SECRET_KEY, NOOBAA_ACCESS_KEY, S3_ENPOINT exported in your notebook environment before uploading to s3.
e.g.
S3_ENDPOINT = https://s3-openshift-storage.apps.sno.sandbox.opentlc.com:443

In [2]:
import os

os.system("""
    alias s3='AWS_ACCESS_KEY_ID=$NOOBAA_ACCESS_KEY AWS_SECRET_ACCESS_KEY=$NOOBAA_SECRET_KEY aws --endpoint $S3_ENDPOINT s3'
    if s3 ls "s3://models" 2>&1 | grep -q 'NoSuchBucket'; then
      s3 mb s3://models
    fi
    s3 cp codellama-7b-instruct.Q8_0.gguf s3://models/codellama-7b-instruct.Q8_0.gguf
""")

upload: ./codellama-7b-instruct.Q8_0.gguf to s3://models/codellama-7b-instruct.Q8_0.gguf


0

Let's try the model out locally first, see what it can do! Make sure transformers library installed - we're using the gpu enabled cuda version.

In [6]:
!pip install ctransformers[cuda]


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Load our model from local disk. Ask it to generate some basic python code.

In [9]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("./codellama-7b-instruct.Q8_0.gguf", local_files_only=True, gpu_layers=-1)

print(llm("generate python code to print Hello World"))

.
'''
from jinja2 import Template

template = """\
# {{ filename }} - Example # {{ i }}
print("Hello, World!")
"""

i = 1
while i < 3:
    with open(f'{i}.py', 'w') as file:
        file.write(Template(template).render(filename=i))
    i += 1



Run the generated code in our notebook ... it runs ! ship it !

In [4]:
from jinja2 import Template

template = """\
# {{ filename }} - Example # {{ i }}
print("Hello, World!")
"""

i = 1
while i < 3:
    with open(f'{i}.py', 'w') as file:
        file.write(Template(template).render(filename=i))
    i += 1

In [7]:
!python 1.py

Hello, World!
